# Data Cleaning

<b>Input</b>:

- Smart Freeways Data 2018 - Links All Days.xlsx <br>
  (14 Links' per minute time series data from 1 Jan 2018 to 25 Oct 2018)
    - Sheet 1: NPI Links - Volume    (data available upto 1 Nov 18)
    - Sheet 2: NPI Links - Speed     (data available upto 25 Oct 18)
    - Sheet 3: NPI Links - Occupancy (data available upto 25 Oct 18)
    
- Minor_Incident_Log2.xlsx <br>
  (Incident Data on Kwinana Fwy from 1 Jan 2018 to 30 Nov 2018)

<b>Output</b>:

- LAD.csv

The new dataframe consists of 298 days $\times$ 1440 mins $\times$ 14 links = 6007680 rows

|ID|DateTime|Length|Volume|Speed|Occupancy|
|--|--------|------|------|-----|---------|
|1-14|2018-01-01 to 2018-10-25||||

- LAD+incident.csv
|ID|DateTime|Length|Volume|Speed|Occupancy|Incident_type|
|--|--------|------|------|-----|---------|--------|
|1-14|2018-01-01 to 2018-10-25|||||


### Data Mapping

Map Link name to ID for convenience sake

| Link ID | NPI Link Name |
|---------|---------------|
|1|Kwinana Fwy NB between Kwinana Fwy Nth Bnd H015 Nth Bnd - H018 East Bnd & Kwinana Fwy Nth Bnd H018 W|
|2|Kwinana Fwy NB between Kwinana Fwy Nth Bnd H018 West Bnd - H015 Nth Bnd & Farrington Rd On - H015 Nt|
|3|Kwinana Fwy NB between Farrington Rd On - H015 Nth Bo & H015 Nth Bound - South St Off|
|4|Kwinana Fwy NB between H015 Nth Bound - South St Off & South St On - H015 Nth Bound|
|5|Kwinana Fwy NB between South St On - H015 Nth Bound & H015 Nth Bound - Leach Hwy Off|
|6|Kwinana Fwy NB between H015 Nth Bound - Leach Hwy Off & Leach Hwy West Bound On - H015|
|7|Kwinana Fwy NB between Leach Hwy West Bound On - H015 & Leach Hwy East Bound On - H015|
|8|Kwinana Fwy NB between Leach Hwy East Bound On - H015 & Cranford Av On - H015 Nth Bou|
|9|Kwinana Fwy NB between Cranford Av On - H015 Nth Bou & H015 Sth Bound - H548|
|10|Kwinana Fwy NB between H015 Sth Bound - H548 & Manning Rd - H547 On Kwinana Fwy Nth Bound|
|11|Kwinana Fwy NWB between Manning Rd - H547 On Kwinana Fwy Nth Bound & Canning Hwy - H549 On Kwinana F|
|12|Kwinana Fwy NB between Kwinana Fwy (northbound) Bus Ln From Canning Hwy: H013 On To H015 Northbound|
|13|Kwinana Fwy NB between Mill Pt Rd - H500 On Kwinana Fwy Nth Bound & Kwinana Fwy Nth Bound H503 Off -|
|14|Kwinana Fwy NB between Kwinana Fwy Nth Bound H503 Off - Mill Pt Rd & Mitchell Fwy Nth Bound|

### Contents

1. [Smart Freeways Data 2018 - Links All Days.xlsx](#1) <br>
    1.1. [Unpivot data](#1.1) <br>
    1.2. [Merge Volume, Speed and Occupancy](#1.2) <br>
    1.3. [Dealing with missing data: Interpolation() and fillna()](#1.3) <br>
2. [Minor_Incidents_Log2.xlsx](#2) <br>
    2.1. [Merge incident type to Link All Days data by DateTime](#2.1) <br>

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import os
import timeit
import numpy as np
import pandas as pd
import seaborn as sns
from math import sqrt
from datetime import date
import holidays
sns.set()
import warnings
warnings.filterwarnings("ignore")

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.sans-serif"] = "Verdana"
mpl.rcParams["lines.markersize"] = 20
# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

## 1. Smart Freeways Data 2018 - Links All Days.xlsx <a class="anchor" id="1"></a>

In [2]:
df = pd.read_excel("data/Smart Freeways Data 2018 - Links All Days.xlsx",
                   sheet_name='NPI Links - Volume', skiprows=2, header=0)
df2 = pd.read_excel("data/Smart Freeways Data 2018 - Links All Days.xlsx",
                   sheet_name='NPI Links - Speed', skiprows=2, header=0)
df3 = pd.read_excel("data/Smart Freeways Data 2018 - Links All Days.xlsx",
                   sheet_name='NPI Links - Occupancy', skiprows=2, header=0)

### 1.1. Unpivot dataframe <a class="anchor" id="1.1"></a>

In [3]:
cols1 = df.columns[:3]
cols2 = [d.strftime('%d %b %y') for d in pd.date_range(start='2018-01-01',
                                               end='2018-10-25',
                                               freq='D')]
cols = [*cols1, *cols2]; cols

['Row Labels',
 'NPI Link Length',
 'Hour Formatted',
 '01 Jan 18',
 '02 Jan 18',
 '03 Jan 18',
 '04 Jan 18',
 '05 Jan 18',
 '06 Jan 18',
 '07 Jan 18',
 '08 Jan 18',
 '09 Jan 18',
 '10 Jan 18',
 '11 Jan 18',
 '12 Jan 18',
 '13 Jan 18',
 '14 Jan 18',
 '15 Jan 18',
 '16 Jan 18',
 '17 Jan 18',
 '18 Jan 18',
 '19 Jan 18',
 '20 Jan 18',
 '21 Jan 18',
 '22 Jan 18',
 '23 Jan 18',
 '24 Jan 18',
 '25 Jan 18',
 '26 Jan 18',
 '27 Jan 18',
 '28 Jan 18',
 '29 Jan 18',
 '30 Jan 18',
 '31 Jan 18',
 '01 Feb 18',
 '02 Feb 18',
 '03 Feb 18',
 '04 Feb 18',
 '05 Feb 18',
 '06 Feb 18',
 '07 Feb 18',
 '08 Feb 18',
 '09 Feb 18',
 '10 Feb 18',
 '11 Feb 18',
 '12 Feb 18',
 '13 Feb 18',
 '14 Feb 18',
 '15 Feb 18',
 '16 Feb 18',
 '17 Feb 18',
 '18 Feb 18',
 '19 Feb 18',
 '20 Feb 18',
 '21 Feb 18',
 '22 Feb 18',
 '23 Feb 18',
 '24 Feb 18',
 '25 Feb 18',
 '26 Feb 18',
 '27 Feb 18',
 '28 Feb 18',
 '01 Mar 18',
 '02 Mar 18',
 '03 Mar 18',
 '04 Mar 18',
 '05 Mar 18',
 '06 Mar 18',
 '07 Mar 18',
 '08 Mar 18',
 '09 Mar

In [4]:
def unpivot_data(df):
    """
    Function to add missing date columns and unpivot
    Smart Freeways Data 2018 - Links All Days.xlsx data file
    """
    # Missing Row Labels and NPI Link Length filled 
    df[['Row Labels', 'NPI Link Length']] = df[['Row Labels', 'NPI Link Length']].fillna(method='ffill')
    
    # Keep rows that have %H:%M time format
    df = df[df['Hour Formatted'].notna()]
    df = df[df['Hour Formatted'].str.len() <= 5]
    
    # Add 25 Mar 18 and/or 26 Mar 18 if one of either of them do not exist
    df = df.reindex(df.columns.union(cols, sort=False), axis=1)
    
    # Replace 24-27 Mar with 17-20 Mar
    df['24 Mar 18'] = df['17 Mar 18']
    df['25 Mar 18'] = df['18 Mar 18']
    df['26 Mar 18'] = df['19 Mar 18']
    df['27 Mar 18'] = df['20 Mar 18']
    
    # Change date columns' format from %d %b %y to %Y-%m-%d datetime
    aslist = df.columns.tolist()
    aslist[3:] = pd.to_datetime(df.columns[3:])
    df.columns = aslist
    
    # Change Hour Formatted column to datetime object
    df['Hour Formatted'] = pd.to_datetime(df['Hour Formatted'],
                                              format='%H:%M')
    
    # Melt (unpivot) dataframe
    df = df.melt(id_vars=['Row Labels', 'NPI Link Length', 'Hour Formatted'],
                  var_name='DateTime', value_name='Value')
    # Create DateTime column from date columns and HourFormatted
    df['DateTime'] = (pd.to_datetime(df['DateTime']) +  
                       pd.to_timedelta(df.pop('Hour Formatted')
                                       .dt.strftime('%H:%M:%S')))
    # Sort dataframe by Row Labels and DateTime
    df = df.sort_values(['Row Labels', 'DateTime'], 
                          ascending=[True, True])
    df.reset_index(drop=True, inplace=True)
    
    return df

In [5]:
vol_df = unpivot_data(df)  # 1 Jan 2018 - 1 Nov 2018
spe_df = unpivot_data(df2) # 1 Jan 2018 - 25 Oct 2018
occ_df = unpivot_data(df3) # 1 Jan 2018 - 25 Oct 2018

In [6]:
# Rename columns
vol_df.rename({'Row Labels': 'Link',
               'NPI Link Length': 'Length',
               'Value': 'Volume'}, axis=1, inplace=True)
spe_df.rename({'Row Labels': 'Link',
               'NPI Link Length': 'Length',
               'Value': 'Speed'}, axis=1, inplace=True)
occ_df.rename({'Row Labels': 'Link',
               'NPI Link Length': 'Length',
               'Value': 'Occupancy'}, axis=1, inplace=True)

In [7]:
vol_df # 305 days * 1440 mins * 14 Links = 6148800

,Link,Length,DateTime,Volume
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,16.0
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,15.0
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,15.0
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,15.0
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,14.0
...,...,...,...,...
6148795,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:55:00,5.0
6148796,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:56:00,5.0
6148797,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:57:00,5.0
6148798,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-11-01 23:58:00,5.0


In [8]:
spe_df # 298 days * 1440 mins * 14 Links = 6007680

,Link,Length,DateTime,Speed
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,80.000000
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,93.999998
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,93.999998
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,95.000000
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,96.999999
...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:55:00,96.999997
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:56:00,96.999997
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:57:00,94.000001
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:58:00,94.000001


In [9]:
occ_df # 298 days * 1440 mins * 14 Links = 6007680

,Link,Length,DateTime,Occupancy
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,1.8
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,1.8
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,1.8
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,1.8
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,1.8
...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:55:00,0.0
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:56:00,0.0
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:57:00,0.0
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:58:00,0.0


### 1.2. Merge Volume, Speed and Occupancy <a class="anchor" id="1.2"></a>

In [60]:
# Merge volume and speed
dataset = vol_df.merge(spe_df, on=['Link','Length','DateTime'])

# Merge dataset and occupancy
dataset = dataset.merge(occ_df, on=['Link','Length','DateTime'])

In [48]:
dataset

,Link,Length,DateTime,Volume,Speed,Occupancy
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,16.0,80.000000,1.8
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,15.0,93.999998,1.8
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,15.0,93.999998,1.8
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,15.0,95.000000,1.8
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,14.0,96.999999,1.8
...,...,...,...,...,...,...
6007675,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:55:00,5.0,96.999997,0.0
6007676,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:56:00,5.0,96.999997,0.0
6007677,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:57:00,5.0,94.000001,0.0
6007678,Kwinana Fwy NWB between Manning Rd - H547 On K...,405.0,2018-10-25 23:58:00,5.0,94.000001,0.0


In [49]:
# Check all links' name
dataset.Link.unique()

array(['Kwinana Fwy NB between Cranford Av On - H015 Nth Bou & H015 Sth Bound - H548',
       'Kwinana Fwy NB between Farrington Rd On - H015 Nth Bo & H015 Nth Bound - South St Off',
       'Kwinana Fwy NB between H015 Nth Bound - Leach Hwy Off & Leach Hwy West Bound On - H015',
       'Kwinana Fwy NB between H015 Nth Bound - South St Off & South St On - H015 Nth Bound',
       'Kwinana Fwy NB between H015 Sth Bound - H548 & Manning Rd - H547 On Kwinana Fwy Nth Bound',
       'Kwinana Fwy NB between Kwinana Fwy (northbound) Bus Ln From Canning Hwy: H013 On To H015 Northbound',
       'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H015 Nth Bnd - H018 East Bnd & Kwinana Fwy Nth Bnd H018 W',
       'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H018 West Bnd - H015 Nth Bnd & Farrington Rd On - H015 Nt',
       'Kwinana Fwy NB between Kwinana Fwy Nth Bound H503 Off - Mill Pt Rd & Mitchell Fwy Nth Bound',
       'Kwinana Fwy NB between Leach Hwy East Bound On - H015 & Cranford Av On - H015 Nth Bo

In [61]:
# Map Link name to ID for convenience sake
dataset['ID'] = dataset['Link']
dataset = dataset.replace({'ID':{
    'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H015 Nth Bnd - H018 East Bnd & Kwinana Fwy Nth Bnd H018 W': 1,
    'Kwinana Fwy NB between Kwinana Fwy Nth Bnd H018 West Bnd - H015 Nth Bnd & Farrington Rd On - H015 Nt': 2,
    'Kwinana Fwy NB between Farrington Rd On - H015 Nth Bo & H015 Nth Bound - South St Off': 3,
    'Kwinana Fwy NB between H015 Nth Bound - South St Off & South St On - H015 Nth Bound': 4,
    'Kwinana Fwy NB between South St On - H015 Nth Bound & H015 Nth Bound - Leach Hwy Off': 5,
    'Kwinana Fwy NB between H015 Nth Bound - Leach Hwy Off & Leach Hwy West Bound On - H015': 6,
    'Kwinana Fwy NB between Leach Hwy West Bound On - H015 & Leach Hwy East Bound On - H015': 7,
    'Kwinana Fwy NB between Leach Hwy East Bound On - H015 & Cranford Av On - H015 Nth Bou': 8,
    'Kwinana Fwy NB between Cranford Av On - H015 Nth Bou & H015 Sth Bound - H548': 9,
    'Kwinana Fwy NB between H015 Sth Bound - H548 & Manning Rd - H547 On Kwinana Fwy Nth Bound': 10,
    'Kwinana Fwy NWB between Manning Rd - H547 On Kwinana Fwy Nth Bound & Canning Hwy - H549 On Kwinana F': 11,
    'Kwinana Fwy NB between Kwinana Fwy (northbound) Bus Ln From Canning Hwy: H013 On To H015 Northbound': 12,
    'Kwinana Fwy NB between Mill Pt Rd - H500 On Kwinana Fwy Nth Bound & Kwinana Fwy Nth Bound H503 Off -': 13,
    'Kwinana Fwy NB between Kwinana Fwy Nth Bound H503 Off - Mill Pt Rd & Mitchell Fwy Nth Bound': 14,
}})

,Link,Length,DateTime,Volume,Speed,Occupancy,ID
0,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:00:00,16.0,80.000000,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
1,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:01:00,15.0,93.999998,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
2,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:02:00,15.0,93.999998,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
3,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:03:00,15.0,95.000000,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...
4,Kwinana Fwy NB between Cranford Av On - H015 N...,2840.0,2018-01-01 00:04:00,14.0,96.999999,1.8,Kwinana Fwy NB between Cranford Av On - H015 N...


In [81]:
# Rearrange columns
dataset = dataset[['DateTime', 'ID', 'Link', 'Length',
                   'Volume', 'Speed', 'Occupancy']]
dataset.info()

### 1.3. Dealing with missing data: Interpolation() and fillna() <a class="anchor" id="1.3"></a>

- Assume that volume will never be zero at all time (at least 1), so 0s are replaced with NaNs.
- For small missing data gap (<5 consecutive NaNs), interpolate/extrapolate with hard limit = 5
- For large missing data gap (>=5 consecutive NaNs), fill NaNs with average (mean) values grouped by ID, day of week and timestamp.
- Round Volume and Occupancy values to integer and to 1 decimal place, respectively.

In [65]:
# Change 0s in Volume column to NaNs
dataset['Volume'].replace(0, np.nan, inplace=True)

In [66]:
# Check number of consecutive NaNs
vol_check = (dataset.Volume.isnull().astype(int)
             .groupby(dataset.Volume.notnull().astype(int).cumsum()).sum())
vol_check.value_counts().sort_index()

0       5945934
1          3069
2          7372
3           785
4           500
5           305
6           186
7           156
8            95
9            73
10           71
11           71
12           53
13           25
14           14
15           15
16           13
17           11
18            9
19           12
20            5
21            2
22            3
23            3
24            4
25            2
26            2
27            3
28            2
32            1
33            1
34           13
38            1
60            1
169           1
184           1
186           1
207           3
212           3
218           2
228           2
236           1
237           1
312           1
856          14
1187          1
Name: Volume, dtype: int64

In [83]:
# Multiindex dataframe
dat = dataset.set_index(['ID', 'DateTime'])
dat = dat.sort_index()

# Retrieve only Length, Volume, Speed, and Occupancy
dat = dat.iloc[:,-4:]; dat

Length  Volume      Speed  Occupancy
ID DateTime                                                 
1  2018-01-01 00:00:00   960.0     7.0  96.000000        1.0
   2018-01-01 00:01:00   960.0     6.0  94.999998        1.0
   2018-01-01 00:02:00   960.0     5.0  90.999999        0.8
   2018-01-01 00:03:00   960.0     5.0  94.999997        0.8
   2018-01-01 00:04:00   960.0     5.0  92.999999        1.0
...                        ...     ...        ...        ...
14 2018-10-25 23:55:00   567.0     6.0  82.999997        0.0
   2018-10-25 23:56:00   567.0     6.0  78.000003        0.0
   2018-10-25 23:57:00   567.0     5.0  78.000001        0.0
   2018-10-25 23:58:00   567.0     6.0  74.000000        0.0
   2018-10-25 23:59:00   567.0     7.0  82.999997        0.0

[6007680 rows x 4 columns]

In [84]:
# For small missing data gap (<5 consecutive NaNs),
# interpolate/extrapolate with hard limit = 5
mask = dat.copy()
grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
grp['ones'] = 1
for i in dat.columns:
    mask[i] = (grp.groupby(i)['ones'].transform('count') < 5) | dat[i].notnull()

dat = dat.interpolate().bfill()[mask]

In [91]:
# Only >=5 consecutive NaNs left
(dat.Volume.isnull().astype(int)
 .groupby(dat.Volume.notnull().astype(int).cumsum()).sum()
 .value_counts().sort_index())

0       5979828
5           305
6           186
7           156
8            95
9            73
10           71
11           71
12           53
13           25
14           14
15           15
16           13
17           11
18            9
19           12
20            5
21            2
22            3
23            3
24            4
25            2
26            2
27            3
28            2
32            1
33            1
34           13
38            1
60            1
169           1
184           1
186           1
207           3
212           3
218           2
228           2
236           1
237           1
312           1
856          14
1187          1
Name: Volume, dtype: int64

In [92]:
# Check average (mean) of values grouped by ID, day of week and timestamp
(dat.groupby([dat.index.get_level_values(0), 
             dat.index.get_level_values(1).strftime('%A %H:%M')])
 [['Volume','Speed','Occupancy']].mean())

Volume      Speed  Occupancy
ID DateTime                                       
1  Friday 00:00     2.738095  91.880953   0.342857
   Friday 00:01     2.714286  93.071428   0.319048
   Friday 00:02     2.642857  93.380952   0.288095
   Friday 00:03     2.619048  93.523809   0.323810
   Friday 00:04     2.476190  93.690476   0.338095
...                      ...        ...        ...
14 Wednesday 23:55  6.697674  79.697674   0.418605
   Wednesday 23:56  6.558140  78.534884   0.372093
   Wednesday 23:57  6.465116  79.395349   0.325581
   Wednesday 23:58  6.302326  79.348837   0.302326
   Wednesday 23:59  6.162791  79.093023   0.372093

[141120 rows x 3 columns]

In [94]:
# For big missing data gap (>=5 consecutive NaNs),
# fill with average (mean) of the same day of week and time
# Group by index 0 and index 1 (ID and day of week timestamp)
dat[['Volume','Speed','Occupancy']] = (dat[['Volume','Speed','Occupancy']]
    .fillna(dat.groupby([dat.index.get_level_values(0), 
                         dat.index.get_level_values(1).strftime('%A %H:%M')])
           [['Volume','Speed','Occupancy']].transform('mean')))

In [95]:
# Check number of consecutive NaNs for Volume
(dat.Volume.isnull().astype(int)
 .groupby(dat.Volume.notnull().astype(int).cumsum()).sum()
 .value_counts().sort_index())

0    6007680
Name: Volume, dtype: int64

In [96]:
# Check number of consecutive NaNs for Speed
(dat.Speed.isnull().astype(int)
 .groupby(dat.Speed.notnull().astype(int).cumsum()).sum()
 .value_counts().sort_index())

0    6007680
Name: Speed, dtype: int64

In [97]:
# Check number of consecutive NaNs for Occupancy
(dat.Occupancy.isnull().astype(int)
 .groupby(dat.Occupancy.notnull().astype(int).cumsum()).sum()
 .value_counts().sort_index())

0    6007680
Name: Occupancy, dtype: int64

In [98]:
dat['Volume'] = round(dat['Volume']) # round to nearest integer
dat['Occupancy'] = round(dat['Occupancy']) # round to 1 dp

In [99]:
dat

Length  Volume      Speed  Occupancy
ID DateTime                                                 
1  2018-01-01 00:00:00   960.0     7.0  96.000000        1.0
   2018-01-01 00:01:00   960.0     6.0  94.999998        1.0
   2018-01-01 00:02:00   960.0     5.0  90.999999        1.0
   2018-01-01 00:03:00   960.0     5.0  94.999997        1.0
   2018-01-01 00:04:00   960.0     5.0  92.999999        1.0
...                        ...     ...        ...        ...
14 2018-10-25 23:55:00   567.0     6.0  82.999997        0.0
   2018-10-25 23:56:00   567.0     6.0  78.000003        0.0
   2018-10-25 23:57:00   567.0     5.0  78.000001        0.0
   2018-10-25 23:58:00   567.0     6.0  74.000000        0.0
   2018-10-25 23:59:00   567.0     7.0  82.999997        0.0

[6007680 rows x 4 columns]

In [100]:
# Save data as CSV
dat.to_csv('data/LAD.csv')

## 2. Minor_Incidents_Log2.xlsx <a class="anchor" id="2"></a>

In [106]:
# Read LAD file if have not run from the beginning
df = pd.read_csv("data/LAD.csv", header=0)

In [107]:
df

,ID,DateTime,Length,Volume,Speed,Occupancy
0,1,2018-01-01 00:00:00,960.0,7.0,96.000000,1.0
1,1,2018-01-01 00:01:00,960.0,6.0,94.999998,1.0
2,1,2018-01-01 00:02:00,960.0,5.0,90.999999,1.0
3,1,2018-01-01 00:03:00,960.0,5.0,94.999997,1.0
4,1,2018-01-01 00:04:00,960.0,5.0,92.999999,1.0
...,...,...,...,...,...,...
6007675,14,2018-10-25 23:55:00,567.0,6.0,82.999997,0.0
6007676,14,2018-10-25 23:56:00,567.0,6.0,78.000003,0.0
6007677,14,2018-10-25 23:57:00,567.0,5.0,78.000001,0.0
6007678,14,2018-10-25 23:58:00,567.0,6.0,74.000000,0.0


In [127]:
# Read Incident file
inc_df = pd.read_excel("data/Minor_Incidents_Log2.xlsx",
                   sheet_name='Sheet1', header=0)

In [128]:
# Drop crossRoadRef and rearrange columns
inc_df = inc_df[['ID', 'DateTime', 'Incident_type']]
inc_df

,ID,DateTime,Incident_type
0,14,2018-01-01 08:21:41,Break Down / Tow Away
1,9,2018-01-03 07:41:15,Break Down / Tow Away
2,9,2018-01-04 09:10:22,Break Down / Tow Away
3,7,2018-01-04 21:19:42,Road Crash
4,11,2018-01-09 01:10:00,Debris / Trees / Lost Loads
...,...,...,...
420,14,2018-11-23 09:32:18,Road Crash
421,11,2018-11-26 08:40:15,Break Down / Tow Away
422,11,2018-11-28 01:16:50,Break Down / Tow Away
423,11,2018-11-28 22:34:50,Break Down / Tow Away


In [129]:
df.DateTime = pd.to_datetime(df.DateTime)
inc_df.DateTime = pd.to_datetime(inc_df.DateTime)
inc_df.sort_values('DateTime', inplace=True)

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6007680 entries, 0 to 6007679
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   ID         int64         
 1   DateTime   datetime64[ns]
 2   Length     float64       
 3   Volume     float64       
 4   Speed      float64       
 5   Occupancy  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 275.0 MB


In [131]:
inc_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 425 entries, 0 to 424
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             425 non-null    int64         
 1   DateTime       425 non-null    datetime64[ns]
 2   Incident_type  425 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 13.3+ KB


In [139]:
df.sort_values(['DateTime', 'ID'])

,ID,DateTime,Length,Volume,Speed,Occupancy
0,1,2018-01-01 00:00:00,960.0,7.0,96.000000,1.0
429120,2,2018-01-01 00:00:00,770.0,8.0,75.000002,1.0
858240,3,2018-01-01 00:00:00,520.0,10.0,54.999999,1.0
1287360,4,2018-01-01 00:00:00,1136.0,10.0,93.000002,1.0
1716480,5,2018-01-01 00:00:00,961.0,12.0,93.999997,1.0
...,...,...,...,...,...,...
4291199,10,2018-10-25 23:59:00,460.0,3.0,94.000001,0.0
4720319,11,2018-10-25 23:59:00,405.0,6.0,93.999997,0.0
5149439,12,2018-10-25 23:59:00,4272.0,7.0,90.000001,1.0
5578559,13,2018-10-25 23:59:00,490.0,7.0,81.999998,0.0


### 2.1. Merge incident type to Link All Days data by ID and nearest DateTime <a class="anchor" id="2.1"></a>

In [143]:
df_merged = pd.merge_asof(left=df.sort_values(['DateTime', 'ID']),
              right=inc_df.sort_values(['DateTime', 'ID']),
              on='DateTime', direction='nearest',
              by='ID').sort_values(['ID', 'DateTime']).reset_index()

In [144]:
df_merged

,index,ID,DateTime,Length,Volume,Speed,Occupancy,Incident_type
0,0,1,2018-01-01 00:00:00,960.0,7.0,96.000000,1.0,"Reduced Visibility (fog, smoke over road)"
1,14,1,2018-01-01 00:01:00,960.0,6.0,94.999998,1.0,"Reduced Visibility (fog, smoke over road)"
2,28,1,2018-01-01 00:02:00,960.0,5.0,90.999999,1.0,"Reduced Visibility (fog, smoke over road)"
3,42,1,2018-01-01 00:03:00,960.0,5.0,94.999997,1.0,"Reduced Visibility (fog, smoke over road)"
4,56,1,2018-01-01 00:04:00,960.0,5.0,92.999999,1.0,"Reduced Visibility (fog, smoke over road)"
...,...,...,...,...,...,...,...,...
6007675,6007623,14,2018-10-25 23:55:00,567.0,6.0,82.999997,0.0,Road Crash
6007676,6007637,14,2018-10-25 23:56:00,567.0,6.0,78.000003,0.0,Road Crash
6007677,6007651,14,2018-10-25 23:57:00,567.0,5.0,78.000001,0.0,Road Crash
6007678,6007665,14,2018-10-25 23:58:00,567.0,6.0,74.000000,0.0,Road Crash


In [145]:
df_merged.Incident_type.count()

6007680